In [2]:
!python -m pip install Unidecode

In [3]:
#Import usefull package
import os
import re
import string
import random
import numpy as np
import pandas as pd
from collections import Counter
from unidecode import unidecode
from csv import reader
#Import nltk packages to manipulate text
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix
from nltk.stem.snowball import SnowballStemmer

from nltk import word_tokenize, WordNetLemmatizer, PorterStemmer
from nltk import NaiveBayesClassifier, classify
from nltk import pos_tag
from nltk import ngrams
#***
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw-1.4')

from nltk import word_tokenize,sent_tokenize

# Let's add a path containing some useful nltk data
nltk.data.path += ['/mnt/share/nltk_data']

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

from functools import reduce
from wordcloud import WordCloud

# scikit-learn packages
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import sklearn.metrics

# word2vec
import gensim
from gensim.models import Word2Vec

C:\Users\Meat is delicious\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
[nltk_data] Downloading package punkt to C:\Users\Meat is
[nltk_data]     delicious\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Meat is
[nltk_data]     delicious\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Meat is
[nltk_data]     delicious\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to C:\Users\Meat is
[nltk_data]     delicious\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading pack

In [4]:
path=""
import re
with open(path+'train_set.csv', 'r',encoding='utf-8') as read_obj:
    csv_reader = reader(read_obj)
    order = r'[0-9]'
    Positive=[]
    Negative=[]
    for row in csv_reader :
      row[1]=re.sub(order,' ',row[1])
      row[1]=row[1].lower()
      row[1]=unidecode(row[1])
      if row[0]=='0':
        Negative.append(row[1])
      else:
        Positive.append(row[1])

In [5]:
train_positive, test_positive = train_test_split(Positive, train_size=0.7, test_size=0.3, shuffle=True)
train_negative, test_negative = train_test_split(Negative, train_size=0.7, test_size=0.3, shuffle=True)

train_set = [(tweet,'negative','train') for tweet in train_negative] + [(tweet,'positive','train') for tweet in train_positive]
test_set = [(tweet,'negative','test') for tweet in test_negative] + [(tweet,'positive','test') for tweet in test_positive]

all_tweets=train_set+test_set
random.shuffle(all_tweets)

stoplist = stopwords.words('french')
stop_punctuation = [':', '(', ')', '/', '|', ',',
                    '.', '*', '#', '"', '&', '~',
                    '-', '_', '\\', '@','?','!','\'']

def get_wordnet_pos(pos_tag):
    """
    Modifies pos_tag to get a more general nature of word
    """
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'
def preprocess(sentence):
    """
    Tokenizes, lowers, and stems
    """
    stemmer = SnowballStemmer('french')
    lemmatizer = WordNetLemmatizer()
    tokenized=word_tokenize(sentence)
    POS_tagging=pos_tag(tokenized)
    lemmatized_tweet = [lemmatizer.lemmatize(word[0],get_wordnet_pos(word[1])) for word in POS_tagging]

    return [stemmer.stem(word.lower()) for word in lemmatized_tweet]

def get_features(text):
    return {word: True for word in preprocess(text) if word not in stoplist and word not in stop_punctuation}

all_features = [(get_features(tweet), label, data_set) for (tweet, label,data_set) in all_tweets]

KeyboardInterrupt: 

In [ ]:
def train(features):
    """
    Trains NBC on training set 
    
    Args :
        features (dict) : dictionnary giving words present in the verbatim
        
    Returns :
        train_set : list of verbatims in the train set
        test_set : list of verbatims in the test set
        classifier : trained NBC model
    """
    # initialize training set and test set
    train_set = [(feature[0],feature[1]) for feature in features if feature[2]=='train']
    test_set = [(feature[0],feature[1]) for feature in features if feature[2]=='test']
    print ('Training set size = ' + str(len(train_set)) + ' tweets')
    print ('Test set size = ' + str(len(test_set)) + ' tweets')
    
    # train the classifier
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier
def evaluate(train_set, test_set, classifier):
    """
    Checks how the classifier performs on the training and test sets
    """
    print ('Accuracy on the training set = ' + str(classify.accuracy(classifier, train_set)))
    print ('Accuracy of the test set = ' + str(classify.accuracy(classifier, test_set)))
    return
# Nothing to understand
def most_informative_features_by_label(classifier, label, n=50):
    """
    Returns features and their importance
    """
    cpdist = classifier._feature_probdist
    feature_list = []
    for (fname, fval) in classifier.most_informative_features(n):
        def labelprob(l):
            return cpdist[l, fname].prob(fval)
        labels = sorted([l for l in classifier._labels if fval in cpdist[l, fname].samples()], 
                        key=labelprob)
        if labels[-1] == label:
            feature_list.append(fname)
    return feature_list

In [ ]:
train_set, test_set, classifier = train(all_features)
classifier.show_most_informative_features(10)
evaluate(train_set, test_set, classifier)

In [ ]:
with open(path+'score_set.csv', 'r',encoding='utf-8') as read_obj:
    csv_reader = reader(read_obj)
    Inconnu={}
    for row in csv_reader :
      row[1]=row[1].lower()
      row[1]=unidecode(row[1])
      Inconnu[row[0]]=row[1]


In [ ]:
result={'sentiment':[]}
for key in Inconnu:
  if key!='id':
    prediction=classifier.classify(get_features(Inconnu[key]))
    if prediction=='positive':
      sentiment=1
    else:
      sentiment=0
    result['sentiment']+=[sentiment]

df=pd.DataFrame.from_dict(result)
df.index.name='id'
df.to_csv(path+'out.csv',sep=';')